In [1]:
# !pip install pyscf==2.6.0
# !pip install openfermion==1.6.1
# !echo cuda-quantum | sudo -S apt-get install -y cuda-toolkit-11.8 && python -m pip install cupy==13.1.0
# !pip install git+https://github.com/yfhuang93/ipie/tree/msd_gpu
# !pip install ipie==0.7.0


# Hybrid workflow for molecular simulations

In this tutorial we implement a quantum-classical hybrid workflow for computing the ground state energies of a stronlgy interacting molecular system. The algorithm is made of two parts
1. a variational quantum eigensolver that uses a quantum-number-preserving ansatz proposed by [Anselmetti et al. (2021)](https://doi.org/10.1088/1367-2630/ac2cb3) to produce a quantum trial wave function $|\Psi_T\rangle$. This is implemented via cudaq.
2. an auxiliary-field quantum Monte Carlo that realizes a classical imaginary time evolution and collects the ground state energy estimates

## Variational Quantum Eigensolver (VQE)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyscf import gto, scf, ao2mo, mcscf

from openfermion import jordan_wigner
from openfermion import generate_hamiltonian

from src.vqe_cudaq_qnp import VqeQnp
from src.vqe_cudaq_qnp import get_cudaq_hamiltonian
from src.utils_ipie import get_coeff_wf, gen_ipie_input_from_pyscf_chk

We start by define the structure, the basis set, and the spin of the molecule. We build the molecule object with PySCF and run a preliminary Hartree-Fock computation.

In [ ]:
# Define the molecular structure and the basis set for the molecule
atom = "systems/O3_spin_0/geo.xyz"
basis = "cc-pVQZ"
spin = 0

In [ ]:
# PYSCF 
# Define the molecule 
molecule = gto.M(
    atom=atom,
    spin=spin,
    basis=basis,
    verbose=0
)

mol_nelec = molecule.nelec
hartee_fock = scf.ROHF(molecule)
hartee_fock.chkfile = "scf.chk"
# Run Hartee-Fock
hartee_fock.kernel()

### Hamiltonian preparation for VQE

Since this molecule contains $\sim XXXX$ orbitals and $\sim XXXX$ total electrons, it is impossible to perform a full VQE in the current generation of simulators. Therefore we need to identify an active space with fewer orbitals and electrons that contribute to the the stronlgy interacting part of the whole molecule. We then run a post-HF computation with the PySCF's built-in CASCI method in order to obtain the one-body ($t_{pq}$) and two-body ($V_{prqs}$) integrals that define the molecular Hamiltonian

$$ H= \sum_{pq}t_{pq}\hat{a}_{p}^\dagger \hat {a}_{q}+\sum_{pqrs}  V_{prqs}\hat a_{p}^\dagger \hat a_{q}^\dagger \hat a_{s}\hat a_{r}$$

In [ ]:
# Define the active space 
num_active_orbitals = 6
num_active_electrons = 8

# Run a casci simulation for computing the Hamiltonian in the active space
my_casci = mcscf.CASCI(hartee_fock, num_active_orbitals, num_active_electrons)
ss = (molecule.spin / 2 * (molecule.spin / 2 + 1))
my_casci.fix_spin_(ss=ss)

e_tot, e_cas, fcivec, mo_output, mo_energy = my_casci.kernel()

# Compute the one-body (h1) and two-body integrals (tbi)
h1, energy_core = my_casci.get_h1eff()
h2 = my_casci.get_h2eff()
h2_no_symmetry = ao2mo.restore('1', h2, num_active_orbitals)
tbi = np.asarray(h2_no_symmetry.transpose(0, 2, 3, 1), order='C')

# Compute the Hamiltonian and convert it to a CUDA-Q operator
mol_ham = generate_hamiltonian(h1, tbi, energy_core.item())
jw_hamiltonian = jordan_wigner(mol_ham)
hamiltonian_cudaq, energy_core = get_cudaq_hamiltonian(jw_hamiltonian)

### Run VQE with CUDA-Q

We can now run the quantum-number-preserving VQE using CUDA-Q. We can choose some options (the number of layers, the maximum number of optimization steps) for the VQE. At the end of the VQE, we store the final state vector that will be used in the classical AFQMC computation.

In [ ]:
# Define some options for the VQE
options = {'n_vqe_layers': 1,
           'maxiter': 100,
           'energy_core': energy_core,
           'return_final_state_vec': True}

n_qubits = 2 * num_active_orbitals
vqe = VqeQnp(n_qubits=n_qubits,
             num_active_electrons=num_active_electrons,
             spin=spin,
             options=options)

results = vqe.run_vqe_cudaq(hamiltonian_cudaq)

# Best energy from VQE
energy_optimized = results['energy_optimized']

# Final state vector
final_state_vector = results["state_vec"]

# List with the energies for all the optimization iterations
energies_vqe = results["callback_energies"]

In [ ]:
plt.xlabel("VQE optimization steps")
plt.ylabel("Energy [Ha]")
plt.plot(energies_vqe, label="VQE")
plt.legend()

### Auxiliary field quantum Monte Carlo (AFQMC)

AFQMC is a numerical method for computing relevant properties of strongly interacting molecules. AFQMC is a type of quantum Monte Carlo method that combines the use of random walks with an auxiliary field to simulate the imaginary-time evolution of a quantum system and drive it to the lowest energy state. This method can provide accurate results for ground-state properties of a wide range of physical systems, including atoms, molecules, and solids. One can find a detailed introduction to AFQMC [here](https://www.cond-mat.de/events/correl13/manuscripts/zhang.pdf).

The implementation for AFQMC we use here is the one given by `ipie` that supports both CPUs and GPUs.

In [ ]:
from ipie.config import config
config.update_option("use_gpu", False)
from ipie.hamiltonians.generic import Generic as HamGeneric
from ipie.qmc.afqmc import AFQMC
from ipie.systems.generic import Generic
from ipie.trial_wavefunction.particle_hole import ParticleHole
from ipie.analysis.extraction import extract_observable

In [ ]:
#AFQMC

# Generate the input Hamiltonian for ipie from the chk file from pyscf
ipie_ham  = gen_ipie_input_from_pyscf_chk("scf.chk",
                                          mcscf=True,
                                          chol_cut=1e-5)

h1e, cholesky_vectors, e0  = ipie_ham 

num_basis = cholesky_vectors.shape[1]
num_chol = cholesky_vectors.shape[0]

system = Generic(nelec=mol_nelec)

ham = HamGeneric(
    np.array([h1e, h1e]),
    cholesky_vectors.transpose((1, 2, 0)).reshape((num_basis * num_basis, num_chol)),
    e0,
)
####################

# Build the trial wavefunction from the state vector computed via VQE
wavefunction = get_coeff_wf(final_state_vector,
                            n_active_elec=num_active_electrons,
                            spin=spin
                           )

trial = ParticleHole(
    wavefunction,
    mol_nelec,
    num_basis,
    num_dets_for_props=len(wavefunction[0]),
    verbose=False)  #set to true if needed

trial.compute_trial_energy = True
trial.build()
trial.half_rotate(ham)
####################

In [ ]:
# Set the AFQMC parameters (num_walkers, num_blocks,...) here 
afqmc_msd = AFQMC.build(
    mol_nelec,
    ham,
    trial,
    num_walkers=100,
    num_steps_per_block=25,
    num_blocks=10,
    timestep=0.005,
    stabilize_freq=5,
    seed=96264512,
    pop_control_freq=5,
    verbose=False,
)

# Run the AFQMC
afqmc_msd.run()
afqmc_msd.finalise(verbose=False)

In [ ]:
qmc_data = extract_observable(afqmc_msd.estimators.filename, "energy")
plt.xlabel("AFQMC steps")
plt.ylabel("Energy [Ha]")
plt.plot(qmc_data["ETotal"], label="AFQMC")
plt.legend()

In [ ]:
x = energies_vqe + list(qmc_data["ETotal"])
y = list(range(len(x)))

plt.xlabel("Optimization steps")
plt.ylabel("Energy [Ha]")
plt.plot(y, x, label="VQE + AFQMC")
plt.legend()